<a href="https://colab.research.google.com/github/kofiadom/Flight_Data_Generator/blob/main/generate_flight_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ydata-synthetic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for easydict: filename=easydict-1.10-py3-none-any.whl size=6491 sha256=d9c7c1685ec8157e709c9c7cf1fee630bc799aa524d330aa7a920ee61bdc34cb
  Stored in directory: /root/.cache/pip/wheels/b7/e3/8c/f814820ac7ef5fd96f38759ce0d3dc51166a3169008b31da53
Successfully built easydict
  Attempting uninstall: easydict
    Found existing installation: easydict 1.13
    Uninstalling easydict-1.13:
      Successfully uninstalled easydict-1.13
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.

In [ ]:
import json
import requests
from ydata_synthetic.synthesizers.regular import RegularSynthesizer
from ydata_synthetic.synthesizers import ModelParameters, TrainParameters
import pandas as pd

# PostgreSQL database connection settings
DB_HOST = "localhost"
DB_NAME = "flight_data"
DB_USER = "my_user"
DB_PASSWORD = "password"

# FlightRadar API endpoint
url = "https://flightradar243.p.rapidapi.com/v1/flights/most-tracked"

headers = {
    "X-RapidAPI-Key": "79a1b32ed1mshac16a51ba8187b1p162d29jsn2549136b9466",
    "X-RapidAPI-Host": "flightradar243.p.rapidapi.com"
}

# Retrieve data from API
response = requests.get(url, headers=headers)
response.raise_for_status()  # Raise an exception for 4xx and 5xx status codes
data = response.json()

In [ ]:
num_cols = ['clicks']
cat_cols = ['flight_id', 'flight', 'callsign', 'squawk', 'from_iata', 'from_city', 'to_iata', 'to_city', 'model', 'type']

In [ ]:
if "data" in data and "data" in data["data"]:
        flights = data["data"]["data"]

        # Convert API data to DataFrame
        # df_flights = pd.DataFrame(flights)

In [ ]:
batch_size = 500
epochs = 500+1
learning_rate = 2e-4
beta_1 = 0.5
beta_2 = 0.9

ctgan_args = ModelParameters(batch_size=batch_size,
                             lr=learning_rate,
                             betas=(beta_1, beta_2))

train_args = TrainParameters(epochs=epochs)

In [ ]:
df_flights = pd.DataFrame(flights)
synth = RegularSynthesizer(modelname='ctgan', model_parameters=ctgan_args)
synth.fit(data=df_flights, train_arguments=train_args, num_cols=num_cols, cat_cols=cat_cols)

Epoch: 0 | critic_loss: 5.83352518081665 | generator_loss: 1.8813265562057495
Epoch: 1 | critic_loss: 5.733384609222412 | generator_loss: 1.8073137998580933
Epoch: 2 | critic_loss: 5.554692268371582 | generator_loss: 1.8323571681976318
Epoch: 3 | critic_loss: 5.3142290115356445 | generator_loss: 1.8046255111694336
Epoch: 4 | critic_loss: 5.115668296813965 | generator_loss: 1.82876455783844
Epoch: 5 | critic_loss: 5.001739025115967 | generator_loss: 1.8279818296432495
Epoch: 6 | critic_loss: 4.675438404083252 | generator_loss: 1.7928639650344849
Epoch: 7 | critic_loss: 4.357841968536377 | generator_loss: 1.8160052299499512
Epoch: 8 | critic_loss: 4.105904579162598 | generator_loss: 1.839058518409729
Epoch: 9 | critic_loss: 3.702624559402466 | generator_loss: 1.7560142278671265
Epoch: 10 | critic_loss: 3.523641347885132 | generator_loss: 1.736029028892517
Epoch: 11 | critic_loss: 3.1561243534088135 | generator_loss: 1.7555607557296753
Epoch: 12 | critic_loss: 2.7963385581970215 | generat

In [ ]:
synth_data = synth.sample(10000)
print(synth_data)

NameError: name 'synth' is not defined

In [ ]:
csv_data = pd.DataFrame(synth_data)

In [ ]:
csv_data.to_csv('flightradar_data.csv', index=False)